# Challenge 5: Multi-Source, Multi-Agent
## Introduction

In this part of the challenge you will add another source of data (structured) to the solution.

<div class="alert alert-block alert-warning">
Be Sure you populated correctly the `.env` file for the AZURE_SEARCH_INDEX. 
We are using <a href="https://pypi.org/project/python-dotenv/">python-dotenv</a> to manage our environment variables. It will also make things easier when deploying the application in Azure.

</div>

### Importing the libraries

In [1]:
from langchain_openai import AzureChatOpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain_community.utilities import SQLDatabase
from langchain.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

### Create Environment Variables

**Important:** Make sure you update your `.env` file.

In [2]:
import os, dotenv,sys
dotenv.load_dotenv(override=True)

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../lib')))


# Setup environment
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_MODEL = os.getenv("AZURE_OPENAI_MODEL")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
AZURE_OPENAI_EMBEDDING = os.getenv("AZURE_OPENAI_EMBEDDING")
# Azure Search
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")
AZURE_SEARCH_API_KEY = os.getenv("AZURE_SEARCH_API_KEY")
AZURE_SEARCH_INDEX = os.getenv("AZURE_SEARCH_INDEX")
# Azure AI Document Intelligence
AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT")
AZURE_DOCUMENT_INTELLIGENCE_API_KEY = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_API_KEY")
# Azure Blob Storage
AZURE_STORAGE_CONNECTION_STRING = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
AZURE_STORAGE_CONTAINER = os.getenv("AZURE_STORAGE_CONTAINER")
AZURE_STORAGE_FOLDER = os.getenv("AZURE_STORAGE_FOLDER")
# SQL Database
SQL_SERVER = os.getenv("SQL_SERVER")
SQL_DB = os.getenv("SQL_DB")
SQL_USERNAME = os.getenv("SQL_USERNAME")
SQL_PWD = os.getenv("SQL_PWD")

# Define the questions list (if you are using your own dataset you need to change this list)
QUESTIONS = [
  "What are the revenues of GOOGLE in the year 2009?",
  "What are the revenues and the operative margins of ALPHABET Inc. in 2022 and how it compares with the previous year?",
  "Can you create a table with the total revenue for ALPHABET, NVIDIA, MICROSOFT and APPLE in year 2023?",
  "Can you give me the Fiscal Year 2023 Highlights for APPLE, MICROSOFT and NVIDIA?",
  "Did APPLE repurchase common stock in 2023? create a table of APPLE repurchased stock with date, numbers of stocks and values in dollars.",
  "What is the value of the cumulative 5-years total return of ALPHABET Class A at December 2022?",
  "What was the price of APPLE, NVIDIA and MICROSOFT stock in 23/07/2024?",
  "Can you buy 10 shares of APPLE for me?"
  ]

# Define the System prompt (you need to update this is you are using your own dataset.)
system_prompt_RAG = """ You are a financial assistant tasked with answering questions related to the financial results of major technology companies listed on NASDAQ, \n
specifically Microsoft (MSFT), Alphabet Inc. (GOOGL), Nvidia (NVDA), Apple Inc. (AAPL), and Amazon (AMZN). \n
if you don't find the answer in the context, just say `I don't know.`"""

system_prompt_START = """
  You are an agent that needs analyze the user question. \n
  Question : {input} \n
  if the question is related to stock prices answer with "stock". \n
  if the question is related to information about financial results answer with "rag". \n
  if the question is unclear or you cannot decide answer with "rag". \n
  only answer with one of the word provided.
  Your answer (stock/rag):
  """
system_prompt_SQL = """
  You are a helpful AI assistant expert in querying SQL Database to find answers to user's question about stock prices.
  If you can't find the answer, say 'I am unable to find the answer.'
  """


## Step 1: Import data to the SQL Database

The database provided is an Azure SQL Database.
Upload the data to the SQL Database using the `./data/fsi/db/create_stock_table.sql` script.

The script will create a table named "stock" in the provided database.
The table contains the following columns:
- **Date** DATE
- **CloseLast** DECIMAL(10, 2) 
- **Volume** INT
- **Open** DECIMAL(10, 2)
- **High** DECIMAL(10, 2)
- **Low** DECIMAL(10, 2)
- **Symbol** VARCHAR(10)

**NOTE**: Be sure that your IP address is not allowed to access the server.  To enable access, use the Azure Management Portal.
It may take up to five minutes for this change to take effect

<div class="alert alert-block alert-warning">
After the Step will be completed, you don't need to run this anymore. it's a one time step as the SQL Table has been already populated. 
</div>

In [3]:
# Create the Connection to the SQL Server Database (sqlalchemy)
engine = create_engine(f"mssql+pymssql://{SQL_USERNAME}:{SQL_PWD}@{SQL_SERVER}:1433/{SQL_DB}")

In [ ]:
# Create the Stock Table in the SQL Server Database
with open('../../data/fsi/db/create_stock_table.sql', 'r') as file:
    sql_statements = file.read()

# Execute the SQL Statements
with engine.connect() as connection:
    for command in sql_statements.split('GO\n'):
        command = command.strip()
        if command:
            connection.execute(text(command))
    connection.execute(text("commit"))

## Step 2: Create a LangChain SQL Agent

In [11]:
# Create the Azure OpenAI Chat Client
llm = AzureChatOpenAI(
    azure_deployment=AZURE_OPENAI_DEPLOYMENT_NAME,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    temperature=0,
    max_retries=2
)

In [12]:
# Create the LangChain SQL Database Object and the SQL Database Toolkit Object to be used by the agent.
# Create the Connection to the SQL Server Database (sqlalchemy)
engine = create_engine(f"mssql+pymssql://{SQL_USERNAME}:{SQL_PWD}@{SQL_SERVER}:1433/{SQL_DB}")

db = SQLDatabase(engine=engine)
stock_toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [5]:
# Create the stock_agent using the Langhcain SQL Agent Class (create_sql_agent)

stock_agent = create_sql_agent( llm=llm,
                                toolkit=stock_toolkit,
                                agent_type="openai-tools",
                                agent_name="StockAgent",
                                verbose=True,
                                agent_executor_kwargs=dict(handle_parsing_errors=True))

In [6]:
# Structure the final prompt from the ChatPromptTemplate
# Include the system message (provided at the biginning of the chapter - system_prompt_SQL) and the user message
sql_prompt = ChatPromptTemplate.from_messages(
    [("system",system_prompt_SQL),
     ("user","{question}\n ai:")])

In [8]:
# Test the agent
print(QUESTIONS[6])
response = stock_agent.invoke(sql_prompt.format(question=QUESTIONS[6]))
print(response['output'])

What was the price of APPLE, NVIDIA and MICROSOFT stock in 23/07/2024?


> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


stock
Invoking: `sql_db_schema` with `{'table_names': 'stock'}`



CREATE TABLE stock (
	[Date] DATE NULL, 
	[CloseLast] DECIMAL(10, 2) NULL, 
	[Volume] INTEGER NULL, 
	[Open] DECIMAL(10, 2) NULL, 
	[High] DECIMAL(10, 2) NULL, 
	[Low] DECIMAL(10, 2) NULL, 
	[Symbol] VARCHAR(10) COLLATE SQL_Latin1_General_CP1_CI_AS NULL
)

/*
3 rows from stock table:
Date	CloseLast	Volume	Open	High	Low	Symbol
2024-08-22	224.53	43695320	227.79	228.34	223.90	AAPL
2024-08-21	226.40	34765480	226.52	227.98	225.05	AAPL
2024-08-20	226.51	30299030	225.77	227.17	225.45	AAPL
*/
Invoking: `sql_db_query_checker` with `{'query': "SELECT Date, CloseLast, Symbol FROM stock WHERE Symbol IN ('AAPL', 'NVDA', 'MSFT') AND Date = '2024-07-23'"}`


```sql
SELECT Date, CloseLast, Symbol 
FROM stock 
WHERE Symbol IN ('AAPL', 'NVDA', 'MSFT') 
AND Date = '2024-07-23'
```


## Step 3: Create the Multi Agent Solution

In [13]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../lib')))

## Import the necessary libraries
from typing import Annotated, Sequence
from pydantic import BaseModel, Field
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_community.retrievers import AzureAISearchRetriever
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from langgraph.prebuilt import tools_condition
from langgraph.graph import StateGraph, END
from its_a_rag import ingestion

In [14]:
# Create the Agent State Class to store the messages between the agents
# this should include the input, output and decision as strings
class AgentState(TypedDict):
    input: str
    output: str
    decision: str
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [15]:
# Create the start_agent that analyze the user question and decide if the question is related to stock prices or financial results
def start_agent(state):
    # Import the LLM (you can use "global" to import the LLM in previous step to avoid re-creating the LLM objects)
    global llm
    start_agent_llm = llm
    # Prepare the prompt for the agent
    # Prompt Example: You are an agent that needs analyze the user question. \n Question : {input} \n if the question is related to stock prices answer with "stock". \n if the question is related to information about financial results answer with "rag". \n if the question is unclear or you cannot decide answer with "rag". \n only answer with one of the word provided. Your answer (stock/rag):
    start_prompt = ChatPromptTemplate.from_messages(
        [("system", system_prompt_START)])

    # Prepare the chain to be executed
    chain = start_prompt | start_agent_llm
    # invoke the chain
    response = chain.invoke({"input": state["input"]})
    # take the decision from the response
    decision = response.content.strip().lower()
    # Return the response for the next agent (decision and input required coming fron the Agent State)
    return {"decision": decision, "input": state["input"]}


test_start_state=AgentState(input=QUESTIONS[6], output="", decision="", messages=[])
result=start_agent(test_start_state)
print(result)
test_start_state = AgentState(
    input=QUESTIONS[2], output="", decision="", messages=[])
result = start_agent(test_start_state)
print(result)

{'decision': 'stock', 'input': 'What was the price of APPLE, NVIDIA and MICROSOFT stock in 23/07/2024?'}
{'decision': 'rag', 'input': 'Can you create a table with the total revenue for ALPHABET, NVIDIA, MICROSOFT and APPLE in year 2023?'}


In [ ]:
## Stock Agent

def stock_agent(state):
    from langchain.agents import AgentType
    # Import the LLM (you can use "global" to import the LLM in previous step to avoid re-creating the LLM objects)
    global llm

    # Create the SQL Database Object and the SQL Database Toolkit Object to be used by the agent.
    engine = create_engine(
        f"mssql+pymssql://{SQL_USERNAME}:{SQL_PWD}@{SQL_SERVER}:1433/{SQL_DB}")

    db = SQLDatabase(engine=engine)
    stock_toolkit = SQLDatabaseToolkit(db=db, llm=llm)
    # Create the agent using the Langhcain SQL Agent Class (create_sql_agent)
    stock_agent = create_sql_agent(llm=llm,
                                toolkit=stock_toolkit,
                                agent_type="openai-tools",
                                agent_name="StockAgent",
                                agent_description="This agent is an expert in querying SQL Database to find answers to user's question about stock prices.",
                                agent_version="0.1",
                                agent_author="itsarag",
                                # verbose=True,
                                agent_executor_kwargs=dict(handle_parsing_errors=True))
    # Structure the final prompt from the ChatPromptTemplate
    sql_prompt = ChatPromptTemplate.from_messages(
        [("system", system_prompt_SQL),
        ("user", "{question}\n ai:")])
    # Prepare the response using the invoke method of the agent
    response = stock_agent.invoke(sql_prompt.format(
        question=state["input"]))
    # Return the response for the next agent (output and input required coming fron the Agent State)
    return {"output": response['output'], "input": state["input"]}

test_stock_state = AgentState(input=QUESTIONS[6], output="", decision="", messages=[])
result = stock_agent(test_stock_state)
print(result)

On 23/07/2024, the stock prices were as follows:
- APPLE (AAPL): $225.01
- NVIDIA (NVDA): $122.59
- MICROSOFT (MSFT): $444.85
{'output': 'On 23/07/2024, the stock prices were as follows:\n- APPLE (AAPL): $225.01\n- NVIDIA (NVDA): $122.59\n- MICROSOFT (MSFT): $444.85', 'input': 'What was the price of APPLE, NVIDIA and MICROSOFT stock in 23/07/2024?'}


In [18]:
## Node rag (this is a clean implementation of the RAG Agent completed in Challenge 4)
def rag_agent(state):
    # Import the LLM (you can use "global" to import the LLM in previous step to avoid re-creating the LLM objects)
    global llm
    rag_agent_llm = llm
    # Define the index (use the one created in the previous challenge)
    retriever_multimodal = AzureAISearchRetriever(
        index_name=AZURE_SEARCH_INDEX, api_key=AZURE_SEARCH_API_KEY, service_name=AZURE_SEARCH_ENDPOINT, top_k=5)
    # Define the chain (as it was in the previous challenge)
    chain_multimodal_rag = (
        {
            "context": retriever_multimodal | RunnableLambda(ingestion.get_image_description),
            "question": RunnablePassthrough(),
        }
        | RunnableLambda(ingestion.multimodal_prompt)
        | rag_agent_llm
        | StrOutputParser()
    )
    # prepare the response using the invoke method of the agent
    response = chain_multimodal_rag.invoke({"input": state["input"]})
    # Return the response for the next agent (output and input required coming from the Agent State)
    return {"output": response, "input": state["input"]}


print(QUESTIONS[4])
test_rag_state = AgentState(input=QUESTIONS[4], output="", decision="", messages=[])
result = rag_agent(test_rag_state)
print(result)

Did APPLE repurchase common stock in 2023? create a table of APPLE repurchased stock with date, numbers of stocks and values in dollars.
{'output': 'Yes, Apple repurchased common stock in 2023. Here is a table summarizing the repurchased stock:\n\n| Date       | Number of Stocks (in thousands) | Value (in dollars) |\n|------------|---------------------------------|---------------------|\n| 2023       | 471,419                         | $76.6 billion       |\n\nNote: The exact dates of the repurchases are not provided in the context.', 'input': 'Did APPLE repurchase common stock in 2023? create a table of APPLE repurchased stock with date, numbers of stocks and values in dollars.'}


In [38]:
# Create the 3 steps graph that is going to be working in the bellow "decision" condition
# Add nodes (start_agent, stock_agent, rag_agent) and conditional edges where the decision with be stock or rag
def create_graph():
    # Create the Workflow as StateGraph using the AgentState
    workflow = StateGraph(AgentState)
    # Add the nodes (start_agent, stock_agent, rag_agent)
    workflow.add_node("start", start_agent)
    workflow.add_node("stock_agent", stock_agent)
    workflow.add_node("rag_agent", rag_agent)
    # Add the conditional edge from start -> lamba (decision) -> stock_agent or rag_agent
    workflow.add_conditional_edges(
        "start",
        lambda x: x["decision"],
        {
            "stock": "stock_agent",
            "rag": "rag_agent"
        }
    )
    # Set the workflow entry point
    workflow.set_entry_point("start")
    # Add the final edges to the END node
    workflow.add_edge("stock_agent", END)
    workflow.add_edge("rag_agent", END)
    # Compile the workflow
    return workflow.compile()

### Step 4: Test the Solution

In [39]:
## Test Solution

# intantiate the graph (create_graph)
graph = create_graph()

# Use the graph invoke to answer the questions
# Test the graph with various questions

for QUESTION in QUESTIONS:
    print (QUESTION)
    result = graph.invoke({"input": QUESTION})
    print(result["output"])
    print ("------------------")

What are the revenues of GOOGLE in the year 2009?
I don't know. The provided context does not include information about Google's revenues for the year 2009.
------------------
What are the revenues and the operative margins of ALPHABET Inc. in 2022 and how it compares with the previous year?
I don't know. The provided context does not contain specific information about the revenues and operative margins of Alphabet Inc. for 2022 or how they compare with the previous year.
------------------
Can you create a table with the total revenue for ALPHABET, NVIDIA, MICROSOFT and APPLE in year 2023?
I don't have the information needed to create a table with the total revenue for ALPHABET, MICROSOFT, and APPLE in the year 2023. The provided context only includes revenue details for NVIDIA.
------------------
Can you give me the Fiscal Year 2023 Highlights for APPLE, MICROSOFT and NVIDIA?
I don't know the Fiscal Year 2023 highlights for Apple and Microsoft based on the provided context. However, 